# Learning from Unlabeled Data: Rise of Semi-Supervised and Self-Supervised Learning

## Technical requirements

We will use the following as technical requirements to run the code in this chapter:
- Python 3
- pip
- Tensorflow (with CUDA if you want to train models on GPUs)
    - Keras is installed as a dependency to this
- scikit-learn Python library
    - Numpy is installed as a dependency to this
- Jupyter notebook if running the code directly from Jupyter

In [ ]:
! python3 -m pip install --upgrade pip

### For M1+ Macbook (64-bit ARM Based processor)

In [ ]:
! arch -arm64 pip3 install --upgrade pip
! arch -arm64 pip3 install tensorflow
! arch -arm64 pip3 install -U scikit-learn

### For Other Computer Systems

In [ ]:
! pip3 install --upgrade pip
! pip3 install tensorflow
! pip3 install -U scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.cluster import KMeans
import numpy as np

## 1. Introduction to Machine Learning

### 1.1 Building a Machine Learning Model

We usually divide the given data into 2 subsets - one for training and other for testing. Below we will use __[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)__ dataset to build a classifier model and a clustering model.

More information about the dataset could be found at __[Learning Multiple Layers of Features from Tiny Images](https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf)__, Alex Krizhevsky, 2009.

You can see we first load the train and test images and labels and then normalize each image to be in range [-1, 1] before it could be fed into our ML model for training and testing.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images = train_images / 127.5 - 1
test_images = test_images / 127.5 - 1

Next, we need to decide on the right learning method and algorithm that would solve the problem at hand. If the problem is to predict classes for each test image, we would train the model using labels of each image through a convolutional neural network as described below.

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

Next, we would select an appropriate loss function and optimization technique.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Finally, we train the model, validate and test it using the right evaluation metrics.

In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')

We'll get the output as follows for each epoch and final test accuracy.

But if the problem is to create clusters of images that represent the same group of entities, we would not use any labels for that purpose. So we collate training and test images and flatten them so that we can use a K-means clustering algortihm to get image clusters.


In [ ]:
images = np.concatenate((train_images, test_images))
images_flattened = images.reshape(images.shape[0], -1)

kmeans = KMeans(n_clusters=10, random_state=42)
cluster_assignments = kmeans.fit_predict(images_flattened)

print(cluster_assignments[:100])